# 1. 텍스트 데이터 다루기

**텍스트 처리성능에 큰 영향을 주는 다양한 텍스트 데이터 전처리 기법을 소개한다. Word나 형태소 레벨의 tokenizer 및 Subword 레벨 tokenizing 기법(BPE, sentencepiece)을 소개하고 사용법을 익힌다.**

- Contents

|-|목차|⏲ 190분|
|:---:|---|:---:|
|1-1| 들어가며 | 30분|
|1-2| 전처리:자연어의 노이즈 제거  🔒| 20분|
|1-3| 분산표현:바나나와 사과의 관계를 어떻게 표현할까? 🔒| 20분|
|1-4| 토큰화:그녀는? 그녀+는? 🔒| 30분|
|1-5| 토큰화:다른 방법들 🔒| 30분|
|1-6| 코튼에게 의미를 부여하기 🔒| 30분|
|1-7| 마무리 🔒| 30분|

- Author

||내용|
|:---:|---|
|사진|![author](https://d3s0tskafalll9.cloudfront.net/media/core/user/2020/07/09/%EB%AC%B8%EC%84%B1%EC%9B%90.jpeg)|
|이름|문성원|
|아이디|@dev-sngwn|
|소개|자연어 처리가 꿀잼이라는 것을 세상 사람들이 다 알았으면 좋겠어요. 특히나 기계번역에는 푹 빠져있답니다.|

## 1-1. 들어가며

![](https://d3s0tskafalll9.cloudfront.net/media/images/GN-1-L-1.max-800x600_qIdTUHT.png)

우리는 수많은 문장 속에서 살아갑니다. 아침에 읽은 신문 기사도 문장, 친구에게 온 메시지도 문장, 지금 읽고 계신 이 글까지도 필자가 작성한 문장이죠. 이 모든 것들을 일상에서 자연히 발생하여 쓰이는 언어, **자연어(Natural Language)** 라 부릅니다.

그런데, 자연어라는 말의 의미에 대해 깊이 생각해 보셨나요? 그럼 자연어가 아닌 언어에는 무엇이 있을까요? 자연어의 반대말이라면... 인공어(Artificial Language)라고나 할까요? 그런 언어가 생각나시나요? 그렇습니다. 프로그래밍 언어(Programming Language)야말로 대표적인 인공어라고 할 수 있겠네요. 여러분이 인공지능을 위해 공부하는 파이썬 같은 것들 말입니다. 그렇다면 우리가 일상적으로 사용하는 자연어와 프로그래밍 언어 사이의 본질적인 차이가 무엇인지 혹시 아시나요?

대학교에서 전산학을 전공하셨다면 컴파일러 수업을 들으시면서 언어의 문법에 대한 이론을 접해보셨을 것입니다. 여기서 문맥 자유 문법(Context-Free Grammar), 문맥 의존 문법(Context-Sensitive Grammar)이라는 용어가 나옵니다. 들어보신 적 없으셔도 상관없습니다. 이 개념을 깊숙이 다루는 것은 오늘의 주제 범위에서 벗어나는 일이니까요. 하지만 자연어처리의 본질적 어려움을 이해해 보는 측면에서 아래 링크의 글들을 읽으면서 한번 개념을 정리해 봅시다.

- [자연 언어 와 프로그래밍 언어, AI Study](https://dukeyang.tistory.com/2)

```
Q1. 자연어와 프로그래밍 언어는 형식언어 이론상 각각 어떤 문법을 따르고 있나요?

예시답안
자연어는 문맥의존 문법(Context-sensitive Grammar), 프로그래밍 언어는 문맥자유 문법(Context-free Grammar)을 따릅니다.
```

```
Q2. 기계적으로 언어를 해석하는 파서를 만들었을 때 해석상 의미의 모호성이 생기지 않으려면 그 언어가 어떤 문법 규칙을 준수해야 하나요?

예시답안
문맥자유 문법(Context-free Grammar)을 따르는 언어여야 모호성이 없이 파싱트리를 만들 수 있습니다.
```

위 두 질문에 대한 답변을 통해 우리는 왜 기계를 통한 자연어처리가 어려운지의 본질적 이유를 파악할 수 있을 것입니다. 프로그래밍 언어에는 파서가 존재합니다. 문법에 맞게 파싱을 했다면 컴퓨터는 우리가 프로그래밍한 내용을 전혀 오해하지 않고 정확하게 우리의 의도대로 동작할 것입니다. 서로 다른 상황(문맥)에서 동작시켜도 같은 동작을 하게 되죠. 모든 프로그래밍 언어는 문맥과 상황에 따라 그 뜻이 달라지지 않는 언어, 다시 말해 문맥을 완전히 배제하고 해석 가능한 언어, 즉 Context-free Grammar를 따르는 언어이기 때문입니다.

그러나 자연어는 이와 다릅니다. 컴퓨터를 통한 자연어처리의 어려움을 보여주는 아래 글을 한번 읽어 봅시다. 이 글은 2016년에 구글에서 선보인 자연어 파서 모델을 소개하면서, 자연어 파싱의 어려움을 함께 설명한 글입니다.

- [Announcing SyntaxNet, Google AI Blog](https://ai.googleblog.com/2016/05/announcing-syntaxnet-worlds-most.html)


```
Q3. 윗글에서 소개된 "Alice drove down the street in her car." 라는 문장은 두가지로 해석될 수 있는 모호성을 가지고 있습니다. 어떻게 해석이 가능한가요?

예시답안
- 앨리스는 자기 차를 운전하여 거리를 달려갔다.
- 앨리스는 자기 차 안에 있는 거리를 운전해 달려갔다.
```

```
Q4. 문법적으로 가능한 위 두 가지 해석 중 어느 것이 맞는지를 파악하려면 어떤 것이 필요하다고 하나요?

예시답안
real world knowledge, 즉 현실 세계에 대한 지식(예를 들어 '자동차 안에 거리가 있을 리는 없다')이 필요합니다.
```


하지만 만약 위 예시 문장의 Alice가 이상한 나라의 앨리스의 Alice라면 어떨까요? 어쩌면 자동차 안에 사람이 다니는 거리가 있을 수도 있지 않을까요?😜 이렇듯 문맥에 따라 얼마든지 해석이 달라질 수 있는 것이 자연어입니다. 그러므로 멋진 자연어처리 모델을 만들려면, 단순히 언어의 문법만으로 충분하지 않고 그 언어의 의미까지 이해하는 과정이 필요합니다. 우리가 앞으로 도전하게 될 자연어처리는 이렇게 만만치 않은 작업입니다.

어쩌면 여러분들은 워드 벡터 기법들을 다루어 본 경험이 이미 있을 수도 있을 텐데요. 워드 벡터는 단어 속에 담긴 의미를 어떤 의미 벡터 공간에 매핑하는 기법이었습니다. 사람이 프로그래밍을 통해 파서를 만드는 방식으로는 도저히 해낼 수 없었던 자연어처리 태스크가, 이러한 머신러닝 기법을 통해 단어 속의 의미를 파악하는 과정을 통해 훨씬 정확하게 처리될 수 있는 가능성이 생겼습니다.

하지만 오늘 다루게 될 내용은 자연어를 그렇게 단어의 의미 단위로 쪼개 내는 작업 자체도 만만치 않다는 것을 보여줄 것입니다. 자연어를 의미 단위로 쪼개는 **토큰화(Tokenization)** 기법은 자연어처리 모델의 성능에 결정적 영향을 미칩니다. 토크나이저가 문장을 단어로 쪼개는 방식에 따라 같은 문장도 완전히 다른 워드 벡터 리스트가 되기 때문입니다. 특히 Wordpiece Model같은 Subword level의 처리 기법은 최신 자연어처리 모델에 필수적으로 자리 잡고 있으므로, 이러한 아이디어가 나오기까지의 자연어 전처리, 토큰화 과정을 이해하는 것은 매우 중요합니다.

앞으로의 코스들을 진행하는 데에 필수적인 테크닉들이니 오늘의 내용을 반드시 숙지하시길 바랍니다!

학습 목표


---


- 분산 표현에 대한 직관적 이해를 얻는다.
- 문장 데이터를 정제하는 방법을 배운다.
- 토큰화의 여러 가지 기법들을 배운다.
- 단어 Embedding을 구축하는 방법에 대해 가볍게 맛본다.

준비물


---


먼저 아래와 같이 작업디렉토리를 구성해 봅시다.
```
$ mkdir -p ~/aiffel/text_preprocess
```

## 1-2. 전처리 : 자연어의 노이즈 제거

**자연어**의 정의는 서론에서 배웠습니다. 그렇다면 **자연어에는 노이즈가 많다**는 말이 이해가 가실까요? 예시를 하나 보겠습니다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/GN-1-L-2.max-800x600_Hcen3qw.png)


이상적인 데이터의 형태는 이른바 **교과서적인 문장들**입니다. 앞으로 배워나가겠지만, 우리가 다루게 될 언어 모델(Language Model)이란 바로 단어가 출현하게 될 확률 모델로 이루어져 있습니다. 그러므로 언어 출현 확률의 일정한 패턴을 통계적으로든 딥러닝을 이용해서든 학습시키려면 문법이나 언어 사용 관습의 일정한 패턴을 따른 텍스트 데이터가 이상적입니다.

하지만 위 예시와 같은 채팅 데이터는 띄어쓰기와 맞춤법, 약어 사용 등에서 문제가 많죠. 우리가 사용하는 일상어들은 교과서적인 문장에서 **예외적으로 변형될 여지가 무궁무진**합니다. 이러한 변형들이 자연어처리 모델 입장에서는 노이즈가 됩니다. 따라서 **아직은** 소설책이나 신문 기사같이 맞춤법이 비교적 정확하고 노이즈가 적은 데이터를 사용하는 것이 일반적입니다. 혹은 여러 가지 테크닉으로 노이즈를 제거한 후 사용하는 방법이 있죠!

**아직은** 이라고 표현한 이유는, 위와 같은 데이터들도 양이 무지막지하게 받쳐준다면 모든 변형을 커버할 수 있기 때문입니다. 심지어 교과서만 읽은 인공지능보다 따분하지 않은 언어력을 가질 수 있게 되겠죠! 다만 지금의 인공지능 수준은 서비스 목적에 맞게 데이터를 골라 사용하는 정도기에 **아직은** 이라는 표현이 적합하다고 봅니다.

그럼 채팅 데이터와 같이 이상적이지 않은 데이터에서 발견할 수 있는 노이즈에는 어떤 유형들이 있을까요?



<br><br>
**불완전한 문장으로 구성된 대화의 경우**<br><br>

한 문장씩 주고 받는 대화와 달리 메신저는 한 문장을 여러 번에 나눠 전송하거나 여러 문장을 한 번에 전송하는 경우가 있습니다.

예1) A: "아니아니" "오늘이 아니고" "내일이지" / B: "그럼 오늘 사야겠네. 내일 필요하니까?"
<br><br>
**문장의 길이가 너무 길거나 짧은 경우**<br><br>

아주 짧은 문장은 의미가 없을 수 있고, 대체로 사용빈도가 높은 리액션에 해당하는 경우가 많아서 언어 모델을 왜곡시킬 우려가 있기 때문에 제외해 주는 게 좋습니다.

예1) A: "ㅋㅋㅋ", "ㅠㅠㅠ"

아주 긴 문장은 대화와는 관계가 없는 문장일 수 있습니다.

예2) A: "이 편지는 영국에서부터 시작되어…"
<br><br>
**채팅 데이터에서 문장 시간 간격이 너무 긴 경우**<br><br>

메신저는 누가 자판을 치는지 모르기 때문에 서로의 말이 얽히게 됩니다. 따라서 서로의 말의 텀이 짧으면 그것은 대화가 아니라 서로 할말만 하는 상태일 수 있습니다.

예1) A: "겨울왕국2" / B: "보러가자" / A: "엊그제 봤는데 잼씀" / B: "오늘 저ㄴ 아니 ㅡㅡ"

혹은 말의 텀이 너무 길다면 그것은 연속된 대화로 보기 어렵습니다.

예2) A: "나 10만원만 빌려줄 수 있어?" / …… / B: "아 미안 지금 봤다 아직 필요해?"
<br><br>
**바람직하지 않은 문장의 사용**<br><br>

욕설의 비율이나, 오타의 비율이 높은 문장은 자연어 모델 학습에 사용하지 않는 것이 좋습니다.



---

어떻습니까? 텍스트 데이터의 특성에 따라 고려해야 할 노이즈의 특성이 무궁무진하지 않나요? 우리가 **아직은** 이런 특성들을 노이즈로 취급해서 제거하고 싶어 하지만, 사실은 이 노이즈에 내포된 미묘한 느낌의 차이나 특수한 상황 정보들 또한 언젠가는 세밀하게 고려할 수 있을 때가 올 것이라 생각합니다.

하지만 지금은 좀 더 단순하고 근본적인 노이즈에 집중하겠습니다. 예를 들면 아래와 같은 문장들이죠.

1. Hi, my name is John. ("Hi," "my", ..., "John." 으로 분리됨) - 문장부호
2. First, open the first chapter. (First와 first를 다른 단어로 인식) - 대소문자
3. He is a ten-year-old boy. (ten-year-old를 한 단어로 인식) - 특수문자

대표적인 세 가지 노이즈 유형입니다. 하나하나 해결하며 완벽한 말뭉치를 만들어봅시다!



노이즈 유형 (1) 문장부호 : Hi, my name is john.


---


먼저 **노이즈 유형 (1) 문장부호**입니다. 우리는 문장부호를 배웠으니 ```Hi,``` 가 ```Hi``` 와 ```,``` 의 결합인 것을 알지만 컴퓨터는 명시해 주지 않는다면 알파벳에 , 가 포함되어 있다고 생각할 수도 있겠죠? 문장부호를 단어와 분리하면 해결이 되는 상황이기 때문에 **문장부호 양쪽에 공백을 추가**하는 방법을 취합시다!

In [ ]:
def pad_punctuation(sentence, punc):
    for p in punc:
        sentence = sentence.replace(p, " " + p + " ")

    return sentence

sentence = "Hi, my name is john."

print(pad_punctuation(sentence, [".", "?", "!", ","]))

Hi ,  my name is john . 


Python은 ```replace()``` 라는 직관적인 함수를 지원해 주기에, 이를 활용하면 풀고자 하는 문제를 쉽게 풀어낼 수 있습니다!

단순히 문장부호 양옆에 공백을 추가해 주는 방식은 문제를 야기할 수도 있습니다. 이를테면 "www.semi01sample.com" 라는 구절이 문장에 포함되어 있다면 "www . semi01sample . com" 이라는 요상한 모양새로 변형을 시키겠죠. "Mr.Johnson" 이나 "45,756" 같은 경우도 마찬가집니다.

이런 경우는 불가피한 손실로 취급하고 넘어갑니다. 모든 규칙에 대해 변환을 정의해 줄 수는 없기 때문이죠. 게다가 이러한 예외들은 보통 우리가 수행하고자 하는 목적에 큰 영향을 주지 못합니다. **"Mr / . / Johnson"** 과 **"Barack / Hussein / Obama"** 는 컴퓨터 입장에서 똑같이 세 단어로 이루어진 이름일 뿐이니까요.

노이즈 유형 (2) 대소문자 : First, open the first chapter.


---


그다음은 **노이즈 유형 (2) 대소문자입**니다. 영어에서 발생하는 문제인데, ```First```와 ```first``` 는 같은 의미를 갖고 있음에도 컴퓨터는 ```f```와 ```F```를 다르다고 구분 지어 버릴 수 있겠죠. 이를 방지하기 위해 **모든 단어를 소문자로 바꾸는 방법**을 취하겠습니다.

엄연히 말하면 위 예시에서 ```First```와 ```first```는 부사와 한정사로 구분되어 각각의 용도를 배우면 더 좋을 것 같지만, 앞서 언급했듯이 **아직은** 이른 접근이랍니다!

In [ ]:
sentence = "First, open the first chapter."

print(sentence.lower())

first, open the first chapter.


세상에... 이렇게 간단할 수가요? lower() 함수가 있다면 그 반대도 당연히 있겠죠?

In [ ]:
sentence = "First, open the first chapter."

print(sentence.upper())

FIRST, OPEN THE FIRST CHAPTER.


노이즈 유형 (3) 특수문자 : He is a ten-year-old boy.


---


마지막인 **노이즈 유형 (3) 특수문자**입니다. ```ten-year-old```와 ```seven-year-old```, 그 외의 수많은 나이 표현들을 각각의 단어 취급을 해버리는 일이 벌어지겠죠. 그런 불상사를 방지하기 위해 특수문자를 제거하려 합니다. 하지만 무한한 특수문자를 모두 정의하여 제거할 수는 없겠죠? 따라서 우리는 사용할 알파벳과 기호들을 정의해 이를 제외하곤 모두 제거하도록 하겠습니다.

In [ ]:
import re

sentence = "He is a ten-year-old boy."
sentence = re.sub("([^a-zA-Z.,?!])", " ", sentence)

print(sentence)

He is a ten year old boy.


re 패키지는 정규표현식 사용을 도와주는 패키지입니다. 정규표현식을 사용하면 어떤 문장을 규칙에 따라 일반화하는 것이 굉장히 편해집니다. 대신 처음 접하는 분들께는 기호들이 다소 난해하게 다가오죠... 이에 대한 자세한 설명이 궁금하시다면 아래 페이지를 방문해보세요!

-[Python 정규표현식(Regex)](https://docs.python.org/ko/3/library/re.html)

지금까지 배운 모든 것을 종합하여 문장을 정제하는 함수로 정의한 후, 다음으로 넘어가도록 하죠!

In [ ]:
# From The Project Gutenberg
# (https://www.gutenberg.org/files/2397/2397-h/2397-h.htm)

corpus = \
"""
In the days that followed I learned to spell in this uncomprehending way a great many words, among them pin, hat, cup and a few verbs like sit, stand and walk. 
But my teacher had been with me several weeks before I understood that everything has a name.
One day, we walked down the path to the well-house, attracted by the fragrance of the honeysuckle with which it was covered. 
Some one was drawing water and my teacher placed my hand under the spout. 
As the cool stream gushed over one hand she spelled into the other the word water, first slowly, then rapidly. 
I stood still, my whole attention fixed upon the motions of her fingers. 
Suddenly I felt a misty consciousness as of something forgotten—a thrill of returning thought; and somehow the mystery of language was revealed to me. 
I knew then that "w-a-t-e-r" meant the wonderful cool something that was flowing over my hand. 
That living word awakened my soul, gave it light, hope, joy, set it free! 
There were barriers still, it is true, but barriers that could in time be swept away.
""" 

def cleaning_text(text, punc, regex):
    # 노이즈 유형 (1) 문장부호 공백추가
    for p in punc:
        text = text.replace(p, " " + p + " ")

    # 노이즈 유형 (2), (3) 소문자화 및 특수문자 제거
    text = re.sub(regex, " ", text).lower()

    return text

print(cleaning_text(corpus, [".", ",", "!", "?"], "([^a-zA-Z0-9.,?!\n])"))


in the days that followed i learned to spell in this uncomprehending way a great many words ,  among them pin ,  hat ,  cup and a few verbs like sit ,  stand and walk .  
but my teacher had been with me several weeks before i understood that everything has a name . 
one day ,  we walked down the path to the well house ,  attracted by the fragrance of the honeysuckle with which it was covered .  
some one was drawing water and my teacher placed my hand under the spout .  
as the cool stream gushed over one hand she spelled into the other the word water ,  first slowly ,  then rapidly .  
i stood still ,  my whole attention fixed upon the motions of her fingers .  
suddenly i felt a misty consciousness as of something forgotten a thrill of returning thought  and somehow the mystery of language was revealed to me .  
i knew then that  w a t e r  meant the wonderful cool something that was flowing over my hand .  
that living word awakened my soul ,  gave it light ,  hope ,  joy ,  set it

## 1-3. 분산표현 : 바나나와 사과의 관계를 어떻게 표현할까?


단어의 희소 표현과 분산 표현


---


임베딩 레이어(Embedding Layer)를 통해 단어의 **분산 표현(distributed representation)** 을 구현할 수 있는데요. 임베딩 레이어는 자연어 처리 분야에서 거의 기본에 해당하는 요소입니다. 그럼 **분산 표현**에 대해 알아 보겠습니다.

**분산 표현**과 반대되는 표현으로 **희소 표현(Sparse representation)** 이라는 것이 있습니다. 어느 표현이든 단어를 벡터로 표현하겠다는 점에서는 동일합니다. 하지만, 단어의 의미를 표현하는 접근 방식에서 큰 차이가 있습니다. **희소 표현**방식은 벡터의 각 차원마다 단어의 특정 의미 속성을 대응시키는 방식입니다. 예를 들자면 아래와 같습니다.

사람의 성별을 표현하는 ```남자```와 ```여자```라는 두 단어를 수로 표현하려면 어떻게 하면 될까요? 예컨대 ```남자: [-1], 여자: [1]``` 혹은 ```남자: [1], 여자: [-1]``` 의 형태로 표현해 볼 수 있겠네요. 전자로 생각을 하도록 하죠.

다음으로 등장한 단어는 ```소년```과 ```소녀```입니다. 두 단어는 각각 어린 남자와 여자를 의미하니, 앞서 생성한 **"성별"** 이라는 속성에 **"나이가 어리다"** 라는 속성을 추가해야겠습니다. 즉, ```소년: [-1, -1], 소녀: [1, -1]``` 이 되겠군요! **"나이가 많다"** 라는 속성을 가진 ```할아버지```, ```할머니```는 자동적으로 ```할아버지: [-1, 1], 할머니: [1, 1]``` 이 되는 것도 알 수 있습니다.

잠깐, 그러면 ```아저씨```와 ```아줌마```는 어떻게 표현하죠..? 아하, 각 속성값들이 정수가 아니라 실수라면 표현이 가능하겠네요! 1차적으로 완성된 우리의 희소 표현 방식의 단어 사전은 아래와 같습니다.

```
{
    //     [성별, 연령]
    남자: [-1.0, 0.0], // 이를테면 0.0 이 "관계없음 또는 중립적" 을 의미할 수 있겠죠!
    여자: [1.0, 0.0],
    소년: [-1.0, -0.7],
    소녀: [1.0, -0.7],
    할머니: [1.0, 0.7],
    할아버지: [-1.0, 0.7],
    아저씨: [-1.0, 0.2],
    아줌마: [1.0, 0.2]
}
```
지금 우리가 한 것이 바로 **희소 표현(Sparse representation)** 입니다. 단어를 고차원 벡터로 변환하는 것이죠. 사람을 나이와 성별로 구분하기 위해선 **적어도 2차원이 필요함**을 배웠네요! 2차원이라 뭐가 고차원인지, 뭐가 희소하다는 건지 느낌이 안 오시나요? 그럼 좀 더 진행해 볼까요?

위 예시는 모든 단어들이 **"사람"이라는 속성을 가짐** 을 전제하죠? 하지만 세상엔 ```바나나```도 있고 ```사과```도 있답니다. **"과일인지 아닌지 판단"** 하는 속성을 추가하고, **"색깔"** 속성을 추가해서 이번엔 ```바나나```와 ```사과```를 구분해보죠!

```
{
    //      [성별, 연령, 과일, 색깔]
    남자: [-1.0, 0.0, 0.0, 0.0],
    여자: [1.0, 0.0, 0.0, 0.0],
    사과: [0.0, 0.0, 1.0, 0.5],   // 빨갛게 잘 익은 사과
    바나나: [0.0, 0.0, 1.0, -0.5] // 노랗게 잘 익은 바나나
}
```



어떤가요? 속성의 종류가 늘어나고 워드 벡터의 차원이 따라서 늘어나기 시작하니까 벡터에서 0.0이 자주 나오기 시작하는 것이 보이시나요? 4차원까지는 우리의 직관으로 늘려보았지만, 우리가 이런 방식으로 세상의 모든 단어의 속성을 규명해서 표현하려면 과연 몇 차원의 벡터가 필요할까요? 이번 생에는 어려울지도 모르겠네요. 엄청난 고차원의 벡터를 이런 방식으로 만들어낸다 한들, 워드 벡터의 대부분의 차원은 0.0으로 채워진 희소 표현이 만들어질 것입니다.

이런 방식의 문제가 무엇일까요? 너무 고차원이 필요하고 불필요한 메모리와 연산량이 낭비된다는 점 말고도 중요한 문제점 하나는, 희소 표현의 워드 벡터끼리는 단어들 간의 의미적 유사도를 계산할 수 없다는 점입니다.

두 고차원 벡터의 유사도는 **코사인 유사도(Cosine Similarity)** 를 통해 구할 수 있습니다. 자세한 내용이 궁금하다면 아래 웹페이지를 확인하세요!

-[위키독스: 코사인 유사도](https://wikidocs.net/24603)

```
Q5. 의미속성을 공유하지 않는 아래 두 희소표현 워드 벡터의 코사인 유사도를 구해보세요. word_1 = {남자: [-1.0, 0.0, 0.0, 0.0]}
word_2 = {사과: [0.0, 0.0, 1.0, 0.5]}

예시답안
공유하는 의미속성이 없는 위 두 벡터의 내적은 0이 되므로, 코사인 유사도도 역시 0이 됩니다. 이렇게 되면 희소표현으로 본 두 단어 사이에는 아무런 의미적 유사도가 없게 됩니다.
```

그래서 우린 **Embedding 레이어**를 사용해 **각 단어가 몇 차원의 속성을 가질지 정의**하는 방식으로 단어의 **분산 표현(distributed representation)** 를 구현하는 방식을 주로 사용하게 됩니다. 만약 100개의 단어를 256차원의 속성으로 표현하고 싶다면 Embedding 레이어는 아래와 같이 정의되겠죠.

```
embedding_layer = tf.keras.layers.Embedding(input_dim=100, output_dim=256)
```
위 단어의 **분산 표현**에는 우리가 일일이 정의할 수 없는 어떤 **추상적인 속성**들이 256차원 안에 골고루 분산되어 표현됩니다. 희소 표현처럼 속성값을 임의로 지정해 주는 것이 아니라, 수많은 텍스트 데이터를 읽어가며 적합한 값을 찾아갈 것입니다. 적절히 훈련된 분산 표현 모델을 통해 우리는 단어 간의 의미 유사도를 계산하거나, 이를 feature로 삼아 복잡한 자연어처리 모델을 훈련시킬 수 있게 됩니다.

이에 대한 방법은 후술하도록 하죠!


단어 사전 구성과 활용의 문제


---


하지만 여기서 짚고 넘어가야 할 점이 있습니다. 위에서 Embedding 레이어를 사용해 구현한 분산 표현은 컴퓨터 입장에서는 단어 사전이 될 것입니다. 그런데 만약 사전만 가지고 외국에 나가면 바로 그 나라 말을 알아듣고 해석할 수 있을까요? 아마 단어 단위로 명확하게 끊어서 들리지조차 않을 것이기 때문에 단어 사전이 있다 한들 아무런 소용이 없게 되지 않을까요?

컴퓨터한테도 비슷한 일이 벌어지곤 합니다. 우리는 마치 문장이 단어 단위로 명확하게 쪼개져 있어서, 컴퓨터가 각 단어에 해당하는 분산 표현을 언제든 정확히 찾을 수 있을 것처럼 전제하고 위와 같은 컴퓨터용 단어 사전을 구축해 두었지만, 정작 컴퓨터는 전혀 엉뚱한 단어로 해석하거나 혹은 사전에서 그 단어를 찾지 못해 당황하는 일이 생기게 됩니다. 그것은 바로 컴퓨터가 문장을 단어 단위로 정확하게 끊어 읽지 못하기 때문에 벌어지는 일입니다. 다음 스텝에서 이 문제의 어려움을 좀 더 자세하게 다루어 봅시다.

## 1-4. 토큰화 : 그녀는? 그녀+는?
한 문장에서 단어의 수는 어떻게 정의할 수 있을까요? ```"그녀는 나와 밥을 먹는다"``` 라는 문장이 주어지면 공백 기준으로 나누어 ```1: 그녀는``` ```2: 나와``` ```3: 밥을``` ```4: 먹는다``` 4개 단어로 이루어졌다고 단정 지을 수 있을까요? 어쩌면 ```1: 그녀``` ```2: 는``` ```3: 나``` ```4: 와``` ```5: 밥``` ```6: 을``` ```7: 먹는다``` 처럼 잘게 잘게 쪼개어 7개 단어로 이루어졌다고 할 수도 있겠죠? 그것은 우리가 정의할 **토큰화 기법이 결정할 부분**이랍니다!

문장을 어떤 기준으로 쪼개었을 때, 쪼개진 각 단어들을 **토큰(Token)** 이라고 부릅니다. 그리고 그 쪼개진 기준이 **토큰화(Tokenization) 기법**에 의해 정해지죠. 이번 스텝에서는 토큰화의 여러 가지 기법에 대해 배워보도록 하겠습니다.


공백 기반 토큰화


---


자연어의 노이즈를 제거하는 방법 중 하나로 우리는 ```Hi,``` 를 ```Hi```와 ```,```로 나누기 위해 문장부호 양옆에 공백을 추가해 주었습니다. 그것은 이 **공백 기반 토큰화**를 사용하기 위해서였죠! 당시의 예제 코드를 다시 가져와 공백을 기반으로 토큰화를 진행해 보겠습니다.

In [ ]:
corpus = \
"""
in the days that followed i learned to spell in this uncomprehending way a great many words ,  among them pin ,  hat ,  cup and a few verbs like sit ,  stand and walk .  
but my teacher had been with me several weeks before i understood that everything has a name . 
one day ,  we walked down the path to the well house ,  attracted by the fragrance of the honeysuckle with which it was covered .  
some one was drawing water and my teacher placed my hand under the spout .  
as the cool stream gushed over one hand she spelled into the other the word water ,  first slowly ,  then rapidly .  
i stood still ,  my whole attention fixed upon the motions of her fingers .  
suddenly i felt a misty consciousness as of something forgotten a thrill of returning thought  and somehow the mystery of language was revealed to me .  
i knew then that  w a t e r  meant the wonderful cool something that was flowing over my hand .  
that living word awakened my soul ,  gave it light ,  hope ,  joy ,  set it free !  
there were barriers still ,  it is true ,  but barriers that could in time be swept away . 
"""

tokens = corpus.split()

print("문장이 포함하는 Tokens:", tokens)

문장이 포함하는 Tokens: ['in', 'the', 'days', 'that', 'followed', 'i', 'learned', 'to', 'spell', 'in', 'this', 'uncomprehending', 'way', 'a', 'great', 'many', 'words', ',', 'among', 'them', 'pin', ',', 'hat', ',', 'cup', 'and', 'a', 'few', 'verbs', 'like', 'sit', ',', 'stand', 'and', 'walk', '.', 'but', 'my', 'teacher', 'had', 'been', 'with', 'me', 'several', 'weeks', 'before', 'i', 'understood', 'that', 'everything', 'has', 'a', 'name', '.', 'one', 'day', ',', 'we', 'walked', 'down', 'the', 'path', 'to', 'the', 'well', 'house', ',', 'attracted', 'by', 'the', 'fragrance', 'of', 'the', 'honeysuckle', 'with', 'which', 'it', 'was', 'covered', '.', 'some', 'one', 'was', 'drawing', 'water', 'and', 'my', 'teacher', 'placed', 'my', 'hand', 'under', 'the', 'spout', '.', 'as', 'the', 'cool', 'stream', 'gushed', 'over', 'one', 'hand', 'she', 'spelled', 'into', 'the', 'other', 'the', 'word', 'water', ',', 'first', 'slowly', ',', 'then', 'rapidly', '.', 'i', 'stood', 'still', ',', 'my', 'whole', 'attenti

첫 번째로 배우는 가장 간단한 토큰화 기법이니만큼 사용법도 아주 간단합니다. ```split()``` 함수를 호출하기만 하면 자동으로 토큰들의 List로 만들어주죠.

단어 사전에 위 단어들이 전부 등재되어 있다면야 썩 나쁘지 않겠죠? 사실상 같은 단어인 ```days``` 와 ```day``` 가 구분되어 따로 저장이 되는 정도의 문제는 있겠지만, s 를 모두 한 칸 띌 수도 없는 노릇이기에 불가피한 손실로 취급합니다(후에 등장할 방법에서 해결될 문제입니다!).

형태소 기반 토큰화


---


하지만 우리에겐 영어 문장이 아닌 한국어 문장을 처리할 일이 더 많을 것이고, 한국어 문장은 **공백 기준**으로 토큰화를 했다간 **엉망진창의 단어들이 등장하는 것**을 알 수 있습니다. 문장부호처럼 *"은 / 는 / 이 / 가"* 양옆에 공백을 붙이자구요? 글쎄요... 가로 시작하는 단어만 해도 가면, 가위, 가족, 가수... 의도치 않은 변형이 너무나도 많이 일어날 것 같네요!

이를 어떻게 해결할 수 있을까요? 정답은 **형태소**에 있습니다. 어릴 적 국어 시간에 배운 기억이 새록새록 나시나요? 상기시켜드리면 형태소의 정의는 아래와 같습니다.

(명사) 뜻을 가진 가장 작은 말의 단위.

예를 들어, ```오늘도 공부만 한다``` 라는 문장이 있다면, ```오늘```, ```도```, ```공부```, ```만```, ```한다``` 로 쪼개지는 것이 바로 형태소죠. 한국어는 이를 활용해 토큰화를 할 수 있습니다!

한국어 형태소 분석기에서 대표적인 KoNLPy를 사용하여 봅시다.

-[KoNLPy: 파이썬 한국어 NLP - KoNLPy 0.4.3 documentation](https://konlpy-ko.readthedocs.io/ko/v0.4.3/)



KoNLPy는 내부적으로 5가지의 형태소 분석 Class를 포함하고 있습니다. 형태소 분석기들은 **특수한 문장(띄어쓰기 X / 오탈자) 처리 성능, 속도** 측면에서 **차이**를 보입니다. 천하무적인 것은 (아직은) 없으니, 각 분석기를 직접 테스트해보고 적합한 것을 선택해 사용하면 됩니다.

아래 웹페이지에서 그 실험을 대신해 주었네요. 정말 좋은 참고 자료입니다!

-[한국어 형태소 분석기 성능 비교](https://iostream.tistory.com/144)




```
Q6. 아래 질문에 적합한 분석기를 찾아보도록 합시다!

인턴 M: 100만 문장의 데이터를 형태소 분석하고 싶습니다. 속도 측면에서 가장 뛰어난 분석기를 쓰는 게 좋아 보이는데, 어떤 분석기를 사용하는 게 좋을까요? 추천해주세요!

예시답안
mecab
```

```
Q7. 아래 질문에 적합한 분석기를 찾아보도록 합시다!

팀장 K: 시간은 오래 걸려도 괜찮으니, 우리는 정확도로 승부합시다. 대신 우리가 사용할 SNS 데이터는 맞춤법이 테러블하니까, 띄어쓰기나 오탈자에도 강건한 분석기를 사용해요. 고생~

예시답안
KOMORAN + (mecab, 꼬꼬마)
```

그럼 실제로 한국어 형태소 분석기를 사용하는 비교 실험을 해 봅시다.

우선 한국어 형태소 분석기를 설치하는 과정이 필요하겠군요. 의외로 만만치 않은 작업이 될 수도 있겠습니다만, 앞으로 한국어 자연어처리에 도전하겠다면 이 정도는 필수과정이라고 생각하고 진행해 봅시다.

우선 아래와 같이 konlpy 및 Mecab의 설치 여부를 확인해 봅시다.

In [ ]:
from konlpy.tag import Hannanum,Kkma,Komoran,Mecab,Okt
print("슝")

슝


설치가 완료되었습니다. 그럼 형태소 분석을 진행해 봅시다. 5개의 형태소 분석기가 다음 문장을 어떻게 해석하는지 비교해 보세요.

```
코로나바이러스는 2019년 12월 중국 우한에서 처음 발생한 뒤
전 세계로 확산된, 새로운 유형의 호흡기 감염 질환입니다.
```

In [ ]:
tokenizer_list = [Hannanum(),Kkma(),Komoran(),Mecab(),Okt()]

kor_text = '코로나바이러스는 2019년 12월 중국 우한에서 처음 발생한 뒤 전 세계로 확산된, 새로운 유형의 호흡기 감염 질환입니다.'

for tokenizer in tokenizer_list:
    print('[{}] \n{}'.format(tokenizer.__class__.__name__, tokenizer.pos(kor_text)))

[Hannanum] 
[('코로나바이러스', 'N'), ('는', 'J'), ('2019년', 'N'), ('12월', 'N'), ('중국', 'N'), ('우한', 'N'), ('에서', 'J'), ('처음', 'M'), ('발생', 'N'), ('하', 'X'), ('ㄴ', 'E'), ('뒤', 'N'), ('전', 'N'), ('세계', 'N'), ('로', 'J'), ('확산', 'N'), ('되', 'X'), ('ㄴ', 'E'), (',', 'S'), ('새롭', 'P'), ('은', 'E'), ('유형', 'N'), ('의', 'J'), ('호흡기', 'N'), ('감염', 'N'), ('질환', 'N'), ('이', 'J'), ('ㅂ니다', 'E'), ('.', 'S')]
[Kkma] 
[('코로나', 'NNG'), ('바', 'NNG'), ('이러', 'MAG'), ('슬', 'VV'), ('는', 'ETD'), ('2019', 'NR'), ('년', 'NNM'), ('12', 'NR'), ('월', 'NNM'), ('중국', 'NNG'), ('우', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('에', 'VV'), ('서', 'ECD'), ('처음', 'NNG'), ('발생', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('뒤', 'NNG'), ('전', 'NNG'), ('세계', 'NNG'), ('로', 'JKM'), ('확산', 'NNG'), ('되', 'XSV'), ('ㄴ', 'ETD'), (',', 'SP'), ('새', 'NNG'), ('롭', 'XSA'), ('ㄴ', 'ETD'), ('유형', 'NNG'), ('의', 'JKG'), ('호흡기', 'NNG'), ('감염', 'NNG'), ('질환', 'NNG'), ('이', 'VCP'), ('ㅂ니다', 'EFN'), ('.', 'SF')]
[Komoran] 
[('코로나바이러스', 'NNP'), ('는', 'JX'), ('2019', 'SN'

어떤가요? 완벽한 토크나이저는 역시 존재하지 않는 것 같습니다. 저마다 장단점이 존재하는데, 문장을 바꿔 가며 토크나이저마다의 장단점을 정리해 보는 것도 좋은 경험이 될 것입니다.

사전에 없는 단어의 문제


---


지금까지 배운 공백 기반이나, 형태소 기반의 토큰화 기법들은 모두 **의미를 가지는 단위로 토큰을 생성**합니다. 이 기법의 경우, 데이터에 포함되는 모든 단어를 처리할 수는 없기 때문에 자주 등장한 상위 N개의 단어만 사용하고 나머지는 ```<unk>```같은 **특수한 토큰(Unknown Token)으로 치환**해버립니다. 자원이 무한하고 데이터도 무한하다면 그러지 않아도 되겠지만... 모든 것은 한정되어 있기 때문에 이 또한 불가피한 손실이죠. 하지만 이것은 종종 큰 문제를 야기합니다.

```
코로나바이러스는 2019년 12월 중국 우한에서 처음 발생한 뒤
전 세계로 확산된, 새로운 유형의 호흡기 감염 질환입니다.

→

<unk>는 2019년 12월 중국 <unk>에서 처음 발생한 뒤
전 세계로 확산된, 새로운 유형의 호흡기 감염 질환입니다.
```

만약 위 문장을 영문으로 번역해야 한다면 어떨까요? 핵심인 단어 ```코로나바이러스```와 ```우한```을 모른다면 제대로 해낼 수 있을 리가 없습니다. 이를 **OOV(Out-Of-Vocabulary)** 문제라고 합니다. 이처럼 **새로 등장한(본 적 없는) 단어에 대해 약한 모습**을 보일 수밖에 없는 기법들이기에, 이를 해결하고자 하는 시도들이 있었습니다. 그리고 그것이 우리가 다음 스텝에서 배울, ***Wordpiece Model***이죠!

## 1-5. 토큰화 : 다른 방법들

*Wordpiece Model(WPM)* 은 우리가 접한 적이 있는 아이디어를 기반으로 만들어졌습니다. 두 단어 ```preview```와 ```predict```를 보면 접두어인 ```pre```가 공통되고 있죠? ```pre```가 들어간 단어는 주로 *"미리", "이전의"* 와 연계되는 의미를 가지고 있습니다. 컴퓨터도 두 단어를 따로 볼 게 아니라 ```pre+view```와 ```pre+dict```로 본다면 학습을 더 잘 할 수 있지 않을까요?

이처럼 한 단어를 여러 개의 Subword의 집합으로 보는 방법이 WPM입니다. WPM의 원리를 알기 전, 먼저 알아야 할 것이 바로 **Byte Pair Encoding(BPE)** 입니다.


Byte Pair Encoding(BPE)


---


BPE 알고리즘이 고안된 것은 1994년입니다. 그때는 자연어 처리에 적용하기 위해서가 아니라 데이터 압축을 위해서 생겨났었죠. 데이터에서 **가장 많이 등장하는 바이트 쌍(Byte Pair)** 을 새로운 단어로 치환하여 압축하는 작업을 반복하는 방식으로 동작합니다. 예시는 아래와 같습니다.

```
aaabdaaabac # 가장 많이 등장한 바이트 쌍 "aa"를 "Z"로 치환합니다.
→ 
ZabdZabac   # "aa" 총 두 개가 치환되어 4바이트를 2바이트로 압축하였습니다.
Z=aa        # 그다음 많이 등장한 바이트 쌍 "ab"를 "Y"로 치환합니다.
→ 
ZYdZYac     # "ab" 총 두 개가 치환되어 4바이트를 2바이트로 압축하였습니다.
Z=aa        # 여기서 작업을 멈추어도 되지만, 치환된 바이트에 대해서도 진행한다면
Y=ab        # 가장 많이 등장한 바이트 쌍 "ZY"를 "X"로 치환합니다.
→ 
XdXac
Z=aa
Y=ab
X=ZY       # 압축이 완료되었습니다!

```

아주 직관적인 알고리즘이죠? 이를 토큰화에 적용하자고 제안한 것은 2015년이었습니다. 모든 단어를 문자(바이트)들의 집합으로 취급하여 자주 등장하는 문자 쌍을 합치면, 접두어나 접미어의 의미를 캐치할 수 있고, 처음 등장하는 단어는 문자(알파벳)들의 조합으로 나타내어 **OOV 문제를 완전히 해결**할 수 있다는 것이죠!

비교적 최근의 기술을 소개해드리는 만큼 논문을 함께 첨부합니다.

- [Neural Machine Translation of Rare Words with Subword Units](https://arxiv.org/pdf/1508.07909.pdf)

위 논문은 Python 소스 코드를 함께 제공해 주어 간편하게 실습을 해 볼 수 있습니다. 논문에서 제공해 주는 예제로 동작 방식을 자세히 들여다보죠!

In [ ]:
import re, collections

# 임의의 데이터에 포함된 단어들입니다.
# 우측의 정수는 임의의 데이터에 해당 단어가 포함된 빈도수입니다.
vocab = {
    'l o w '      : 5,
    'l o w e r '  : 2,
    'n e w e s t ': 6,
    'w i d e s t ': 3
}

num_merges = 5

def get_stats(vocab):
    """
    단어 사전을 불러와
    단어는 공백 단위로 쪼개어 문자 list를 만들고
    빈도수와 쌍을 이루게 합니다. (symbols)
    """
    pairs = collections.defaultdict(int)
    
    for word, freq in vocab.items():
        symbols = word.split()

        for i in range(len(symbols) - 1):             # 모든 symbols를 확인하여 
            pairs[symbols[i], symbols[i + 1]] += freq  # 문자 쌍의 빈도수를 저장합니다. 
        
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
        
    return v_out, pair[0] + pair[1]

token_vocab = []

for i in range(num_merges):
    print(">> Step {0}".format(i + 1))
    
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get)  # 가장 많은 빈도수를 가진 문자 쌍을 반환합니다.
    vocab, merge_tok = merge_vocab(best, vocab)
    print("다음 문자 쌍을 치환:", merge_tok)
    print("변환된 Vocab:\n", vocab, "\n")
    
    token_vocab.append(merge_tok)
    
print("Merge Vocab:", token_vocab)

>> Step 1
다음 문자 쌍을 치환: es
변환된 Vocab:
 {'l o w ': 5, 'l o w e r ': 2, 'n e w es t ': 6, 'w i d es t ': 3} 

>> Step 2
다음 문자 쌍을 치환: est
변환된 Vocab:
 {'l o w ': 5, 'l o w e r ': 2, 'n e w est ': 6, 'w i d est ': 3} 

>> Step 3
다음 문자 쌍을 치환: lo
변환된 Vocab:
 {'lo w ': 5, 'lo w e r ': 2, 'n e w est ': 6, 'w i d est ': 3} 

>> Step 4
다음 문자 쌍을 치환: low
변환된 Vocab:
 {'low ': 5, 'low e r ': 2, 'n e w est ': 6, 'w i d est ': 3} 

>> Step 5
다음 문자 쌍을 치환: ne
변환된 Vocab:
 {'low ': 5, 'low e r ': 2, 'ne w est ': 6, 'w i d est ': 3} 

Merge Vocab: ['es', 'est', 'lo', 'low', 'ne']


만일 ```lowest```라는 처음 보는 단어가 등장하더라도, 위 알고리즘을 따르면 어느 정도 의미가 파악된 ```low```와 ```est```의 결합으로 표현할 수 있습니다. 또 BPE의 놀라운 점은 아무리 큰 데이터도 원하는 크기로 OOV 문제없이 사전을 정의할 수 있다는 것입니다. 극단적으로 생각했을 때 알파벳 26개와 특수문자, 문장부호를 아무리 추가해도 100개 이내로 사전을 정의할 수 있죠.(물론 그러면 안 됩니다!!)

Embedding 레이어는 **단어의 개수 x Embedding 차원 수** 의 Weight를 생성하기 때문에 단어의 개수가 줄어드는 것은 곧 메모리의 절약으로 이어집니다. 많은 데이터가 곧 정확도로 이어지기 때문에 이런 기여는 굉장히 의미가 있습니다!

하지만 아직도! 완벽하다고는 할 수 없습니다. 만약 수많은 데이터를 사용해 만든 BPE 사전으로 모델을 학습시키고 문장을 생성하게 했다고 합시다. 그게 ```[i, am, a, b, o, y, a, n, d, you, are, a, gir, l]```이라면, 어떤 기준으로 이들을 결합해서 문장을 복원하죠? 몽땅 한꺼번에 합쳤다간 끔찍한 일이 벌어질 것만 같습니다...

Wordpiece Model(WPM)


---


이에 구글에서 BPE를 변형해 제안한 알고리즘이 바로 WPM입니다. WPM은 BPE에 대해 **두 가지 차별성**을 가집니다.

1. 공백 복원을 위해 단어의 시작 부분에 언더바 _ 를 추가합니다.
2. 빈도수 기반이 아닌 가능도(Likelihood)를 증가시키는 방향으로 문자 쌍을 합칩니다.

첫 번째 문항은 아주 쉬운 내용으로, 앞서 사용한 예문을 빌리면 ```[_i, _am, _a, _b, o, y, _a, n, d, _you, _are, _a, _gir, l]```로 토큰화를 한다는 것입니다. 이렇게 하면 문장을 복원하는 과정이 **1) 모든 토큰을 합친 후, 2) 언더바 _를 공백으로 치환**으로 마무리되어 간편하죠.

두 번째 문항은 다소 난해하게 다가올 수도 있습니다. 직관적인 이해를 얻고 넘어가는 것을 목표로 하죠. 본 내용은 아래 논문 3절과 4절에 자세하게 나와 있습니다.

- [JAPANESE AND KOREAN VOICE SEARCH](https://static.googleusercontent.com/media/research.google.com/ko//pubs/archive/37842.pdf)

(여기서 잠깐) 구글에서 이 기법을 한국어, 일본어 텍스트 처리를 위해 고려했다는 사실이 흥미롭지 않나요? 이 점은 2가지를 시사합니다.

- 조사, 어미 등의 활용이 많고 복잡한 한국어 같은 모델의 토크나이저로 WPM이 좋은 대안이 될 수 있다.
- WPM은 어떤 언어든 무관하게 적용 가능한 language-neutral하고 general한 기법이다. 한국어 형태소 분석기처럼 한국어에만 적용 가능한 기법보다 훨씬 활용도가 크다.

그리고 가능도에 대한 개념이 낯설다면 아래 웹페이지를 방문해보시길 권장합니다.

- [확률(probability)과 가능도(likelihood) 그리고 최대우도추정(likelihood maximazation)](https://jjangjjong.tistory.com/41)

저자들은 *Katz's Back-off Model* 을 통해 토큰화된 사전으로부터 결합할 단어를 선정합니다. *Katz's Back-off Model* 은 LM에서 등장하지 않은 단어에 대해 확률을 할당할 수 있는 모델입니다. 흔히 사용되는 개념은 아니므로 "그런 게 있구나~" 하고 알아만 두시면 좋습니다. 수식을 아래에 첨부하지만 이해하지 않으셔도 괜찮습니다!

![[▲ Katz's Back-off Model]](https://d3s0tskafalll9.cloudfront.net/media/images/GN-1-L-3.max-800x600_uZeTAWo.png)

토큰화의 끝판왕으로 보이는 이 WPM은 아쉽게도 공개되어 있지는 않습니다. 대신에 구글의 *SentencePiece* 라이브러리를 통해 고성능의 BPE를 사용할 수 있습니다! *SentencePiece*에는 전처리 과정도 포함되어 있어서, 데이터를 따로 정제할 필요가 없어 간편하기까지 합니다.

-[google/sentencepiece](https://github.com/google/sentencepiece)

이제 우리는 어떤 언어에도 OOV 발생 우려 없이 안정적으로 활용할 수 있는 멋진 토크나이징 기술을 확보했습니다. 이제는 컴퓨터가 단어사전을 안심하고 활용할 수 있겠군요!

soynlp


---


이외에도 한국어를 위한 토크나이저로 soynlp를 활용할 수 있습니다. soynlp는 한국어 자연어 처리를 위한 라이브러리인데요. 토크나이저 외에도 단어 추출, 품사 판별, 전처리 기능도 제공합니다.

- [soynlp](https://github.com/lovit/soynlp)

형태소 기반의 토크나이저가 미등록 단어에 취약하기 때문에 WordPiece Model을 사용하는 것처럼, 형태소 기반인 koNLPy의 단점을 해결하기 위해 soynlp를 사용할 수 있습니다.

soynlp의 토크나이저는 '학습데이터를 이용하지 않으면서 데이터에 존재하는 단어를 찾거나, 문장을 단어열로 분해, 혹은 품사 판별을 할 수 있는 비지도학습 접근법을 지향합니다' 라고 밝히고 있는데요.

문장에서 처음 단어를 받아들일 때 단어의 경계를 알아야 올바르게 토큰화를 할 수 있습니다. 이때 단어의 경계를 비지도학습을 통해 결정하겠다는 말이에요. 비지도학습을 통한 방법이기 때문에 미등록 단어도 토큰화가 가능합니다. 여기서 비지도학습을 가능케 하는 것이 통계적인 방법이라서 soynlp를 통계 기반 토크나이저로 분류하기도 합니다.

```트와이스```가 한 단어임을 인지하기 위해서 ```트```, ```트와```, ```트와이```, ```트와이스``` 각각 다음 글자의 확률을 계산해서 비교한다고 생각하면 좋습니다. 수학적으로 자세한 내용은 여기서는 다루지 않습니다. 다만 koNLPy외에도 soynlp가 있다는 점을 기억해 주세요.

그럼에도...


---


이렇게 토큰화 방법을 알아 보았지만 이걸 가지고 단어의 분산 표현을 얻는 법을 제대로 다루지는 않았습니다. 이쯤에서 이런 고민이 생기게 됩니다. 한국어라면 ```자동차```를 ```_자동 / 차``` 로 분리되는데... 속성이 아무리 추상적이래도 보기에 ```차```가 마시는 차인지, 달리는 차인지 도통 알 수가 없죠? 게다가 설령 토큰화가 완벽하다고 해도, ```남자```가 ```[-1, 0]```인지 ```[1, 0]```인지는 컴퓨터 입장에서는 알 도리가 없습니다.

Embedding 레이어는 선언 즉시 랜덤한 실수로 Weight 값을 채우고, 학습을 진행하며 적당히 튜닝해가는 방식으로 속성을 맞춰가지만 이는 뭔가 찜찜합니다. 토큰들이 멋지게 의미를 갖게 하는 방법은 없을까요?



## 1-6. 토큰에게 의미를 부여하기
각 토큰들이 랜덤하게 부여된 실수로 살아가지 않게, 그들끼리 유사도 연산을 할 수 있게 의미를 부여하는 알고리즘, 당연히 있습니다! 심지어 토큰화 기법보다 더 많이요! 대표적인 3가지만 알아보도록 하겠습니다. 이번 코스에선 간단하게 아이디어만 이해해도 충분합니다.

Word2Vec


---



*Word2Vec*은 "단어를 벡터로 만든다"는 멋진 이름을 가지고 있습니다. ```난 오늘 술을 한 잔 마셨어``` 라는 문장의 각 단어 즉, **동시에 등장하는 단어끼리는 연관성이 있다**는 아이디어로 시작된 알고리즘입니다. 예문의 경우 다른 단어는 몰라도 ```술```과 ```마셨어```는 괜찮은 연관성을 캐치해낼 수 있겠네요.

Word2Vec에 대한 좋은 자료를 읽어보시고, 다시 얘기해보도록 하죠!

-[위키독스/Word2Vec](https://wikidocs.net/22660)

```
Q8. Word2Vec에는 두 가지 방식이 존재합니다. 각각 무엇인가요?

 
예시답안
예시답안
CBOW, Skip-gram
```
```
Q9. 윗글의 저자에 따르면 Word2Vec은 딥러닝 모델이 아닙니다. 그 이유는 무엇일까요?

 
예시답안
예시답안
은닉층이 1개인 경우 Deep Neural Network가 아닌 Shallow Neural Network라 칭한다.
```
```
Q10. Q8의 두 가지 방식 중, 어느 것이 더 성능이 좋을 것 같은지 생각해봅시다. 그리고 실제로는 어떤지 인터넷에서 찾아 적어봅시다.

 
예시답안
예시답안
(논리적으론 CBOW가 좋아 보이지만) Skip-gram이 실제 실험에선 다소 우세하다.
```

FastText


---


Word2Vec은 정말 좋은 방법이지만, 연산의 빈부격차가 존재했습니다. 자주 등장하지 않는 단어는 최악의 경우 단 한 번의 연산만을 거쳐 랜덤하게 초기화된 값과 크게 다르지 않은 상태로 알고리즘이 종료될 수 있습니다. FastText는 이를 해결하기 위해 BPE와 비슷한 아이디어를 적용했습니다.

자세한 내용은 아래 웹페이지에서 만나보세요! 글 말미의 ```Fasttext의 등장 (2016)``` 파트를 읽으시면 됩니다.

-[한국어를 위한 어휘 임베딩의 개발 -1-](https://brunch.co.kr/@learning/7)

```
Q11. 기존의 단어마다 Embedding을 할당하던 방식과 FastText의 큰 차이점은 무엇인가요? 간단하게 서술해봅시다.


예시답안
FastText는 한 단어를 n-gram의 집합이라고 보고 단어를 쪼개어 각 n-gram에 할당된 Embedding의 평균값을 사용하였다.
```

ELMo - the 1st Contextualized Word Embedding


---


위에 소개했던 Word Embedding 알고리즘들은 (역시나) 정말 훌륭하지만, 여전히 고질적인 문제점이 있습니다. 바로 **고정적**이라는 겁니다! 무슨 말이냐면, 동음이의어를 처리할 수 없다는 얘기입니다.

- 이렇게나 탐스럽고 먹음직스러운 **사과**를 보셨나요?
- 저의 간절한 **사과**를 받아주시기 바랍니다.

우리는 이 두 문장에 나오는 '사과'의 의미가 다르다는 것을 알고 있습니다. 그러나 Word2Vec이든 FastText이든 간에 이 두 문장에 나오는 **사과**의 워드 벡터값은 동일할 수밖에 없습니다.

분명 의미가 다른 두 '사과'의 의미를 명확하게 해석하려면 우리에게 무엇이 필요할까요? 네, 서론에서 우리는 그 답을 미리 살펴보았습니다. Context-sensitive Grammar를 따르는 자연어를 이해하려면 문맥(context)의 활용이 필수적입니다.
여기서 '사과'의 context가 되는 것은 무엇일까요? 첫 문장이라면 ```탐스럽고 먹음직스러운``` 이 될 것이고 다음 문장이라면 ```간절한``` 이 될 것입니다. 즉, 단어의 의미 벡터를 구하기 위해서는 그 단어만 필요한 것이 아니라 그 단어가 놓인 주변 단어 배치의 맥락이 함께 고려되는 Word Embedding이 필요한 것입니다. 이런 개념을 ```Contextualized Word Embedding```이라고 합니다.

2018년 NLP계에 큰 폭풍을 몰고 왔던 *ELMo*라는 모델은 데이터에 단어가 등장한 순간, 그 주변 단어 정보를 사용해 Embedding을 구축하는 개념을 처음 소개하면서 자연어처리의 획기적인 발전의 계기를 마련해 준 첫 번째 ```Contextualized Word Embedding``` 모델입니다. 정말 멋진 아이디어에요! 아래 웹페이지를 방문해 편안하게 글을 읽어보시길 바랍니다.

- [전이 학습 기반 NLP (1): ELMo](https://brunch.co.kr/@learning/12)

```
Q12. 양방향 LSTM을 활용한 ELMo 모델에서 Contextual Word Embedding이 되는 벡터는 어떤 벡터 3가지를 합쳐서 얻어지나요?

 
예시답안
예시답안
기존 어휘 임베딩(입력 토큰의 word vector), 순방향 LSTM의 hidden state vector, 역방향 LSTM의 hidden state vector를 concatenate한 벡터가 ELMo의 Contextual Word Embedding이 됩니다.
```

지금은 모델 구조나 메커니즘을 명확하게 이해하지 않아도 좋습니다. ```Contextualized Word Embedding``` 이라는 것이 있고 ELMo나 BERT 등에 많이 사용되고 있다는 점만 기억해도 충분합니다.

## 1-7. 마무리

![](https://d3s0tskafalll9.cloudfront.net/media/images/GN-1-L-4.max-800x600_PmSMYzs.jpg)


고생 많으셨습니다!

이번에 배운 내용은 토큰화와 분산 표현이 중심입니다. 문장이 입력되면 적절히 토큰화를 하고 토큰을 임베딩(Embedding)을 통해 분산 표현으로 만드는 것이지요. 분산 표현은 벡터이므로 이제 인공지능에 활용할 수 있습니다.

토큰화에 사용되는 방법은 언어마다 다른데요. 문장 구성 성분이 다르기 때문입니다. 조사가 있는 한국어는 형태소 기반인 koNLPy를 주로 쓰고, WordPiece Model인 SentencePiece를 쓸 수도 있어요. 물론 그 외에 다른 방법도 있습니다.

토큰화를 마친 후 임베딩을 할 때는 토큰마다 독립적으로 만들지 않고 토큰 간의 관계성을 주입합니다. 그래야 문장을 구성할 때 적절히 사용될 수 있기 때문이에요. 이렇게 토큰 간의 관계성을 고려해서 만든 것으로는 Word2Vec, FastText 등이 있어요. 거기다가 문장의 문맥까지 고려하는 ELMo까지 등장했습니다.

이제 지금까지 배운 내용이 그려지나요? 오늘 배운 것들은 앞으로 진행될 모든 것들의 기초가 되는 부분이니 **꼭 숙지하시길 권장**합니다.

자연어 처리 분야는 인공지능 모델도 중요하지만 토큰화와 임베딩에 관련된 내용이 매우 중요합니다. 그러니 오늘 배용을 토대로 이 분야에 재미를 느껴 멋진 NLP 연구자로 성장하시기를 바랍니다. 고생하셨습니다!!

